In [ ]:
!pip install transformers accelerate safetensors -q
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [ ]:
model_id = "cxllin/Llama2-7b-med-v1"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id,torch_dtype=torch.float16,device_map="auto",offload_folder="offload_dir",trust_remote_code=True)


In [ ]:
#Lets write all the user inputs

In [ ]:
def get_user_inputs():
  print("Enter the Patient Details:\n")
  patient_name = input("enter patient name  : ")
  patient_weight = input("Enter patient weight in kg : ")
  patient_age = input("Enter patient Age : ")
  symptoms = input("1. Enter Symptoms with comma seperation : ")
  medical_history = input("2. medical history (e.g.,smoker,drinker,have diabetes, etc.,) : ")
  medical_report = input("3. Medical report data (e.g., CRP=10, troponin=0.2, Hb=xyz, etc.,) : ")
  user_query = input("4. What help you need? (e.g., what disease is it can be? should i visit hospital? etc.,) : ")
  return  patient_name,patient_weight,patient_age, symptoms, medical_history, medical_report, user_query

In [ ]:
#Lets write the prompt

In [ ]:
def build_medical_prompt(patient_name,patient_weight,patient_age,symptoms,medical_history,medical_report,user_query):


  prompt = (
      f"Patient name:{patient_name}\n"
      f"Patient Weight: {patient_weight}\n"
      f"patient Age: {patient_age}\n"
      f"Patient Symptoms: {symptoms}\n"
      f"Medical History: {medical_history}\n"
      f"Medical Report: {medical_report}\n"
      f"User Query: {user_query}\n\n"
      "Based on the above, provide:\n"
      "1. Top 4 likely diseases with confidence scores (%) \n\n\n"
      "2. Recommended medicines and treatment\n\n\n"
      "3. Precautions\n\n\n"
      "4. Should the patient go to the hospital? (Yes/No with reason).\n\n\n"
      "5. Finally, explain all this in simple words for the patient to understand clearly.")
  return prompt



In [ ]:
# Lets use LLM 1 Now

In [ ]:
def get_medical_response(prompt,max_tokens=520):
  inputs = tokenizer(prompt,return_tensors="pt").to("cuda")



  output = model.generate(
      **inputs,
      max_new_tokens = max_tokens,
      do_sample = True,
      top_p = 0.9,
      temperature = 0.7,
      repetition_penalty = 1.1,
      eos_token_id = tokenizer.eos_token_id,
      pad_token_id = tokenizer.pad_token_id)

  result = tokenizer.decode(output[0][inputs['input_ids'].shape[1]:],skip_special_tokens=False)

  return result


In [ ]:
import os

In [ ]:
def save_patient_report(patient_name,llm_output):
  folder = "patient_reports"
  os.makedirs(folder, exist_ok=True)
  from datetime import datetime
  date= datetime.today().strftime("%Y-%m-%d")
  filename = f"{patient_name}_{date}.txt"
  file_path = os.path.join(folder,filename)

  with open(file_path,"w",encoding="utf-8") as file:
    file.write(f"Patient Name:{patient_name}\n")
    file.write(f"Date: {date}\n\n")
    file.write("----- Patient Diagnosis Report -----\n\n")
    file.write(llm_output)
  print(f"Report saved as {filename}")

In [ ]:
def gradio_interface(name,weight,age,symptoms,history,report,query):
  prompt = build_medical_prompt(name,weight,age,symptoms,history,report,query)
  response = get_medical_response(prompt)
  saved_file = save_patient_report(name,response)
  return response + f"\n\n\n{saved_file}"

In [ ]:
pip install gradio

In [ ]:
import gradio as gr

In [ ]:
gr.Interface(
    fn=gradio_interface,
    inputs=[
        gr.Textbox(label="Patient Name"),
        gr.Textbox(label="Weight (kg)"),
        gr.Textbox(label="Age"),
        gr.Textbox(label="Symptoms"),
        gr.Textbox(label="Medical History"),
        gr.Textbox(label="Medical Report Values"),
        gr.Textbox(label="User query")],
    outputs = gr.Textbox(label="Diagnosis Result"),
    title = "GenAI Medical Assistant",
    description = "Enter patient details to get disease predictions, Treatments and advices."
    ).launch()

In [ ]:
# def main():

#   patient_name,patient_weight,patient_age, symptoms, medical_history, medical_report, user_query = get_user_inputs()

#   prompt = build_medical_prompt(patient_name,patient_weight,patient_age,symptoms,medical_history,medical_report,user_query)
#   llm_output = get_medical_response(prompt)
#   save_patient_report(patient_name,llm_output)
#   print("\n----  Here we go... Patient Diagnosis REPORT   ----\n")
#   print(llm_output)




# if __name__=="__main__":
#   main()
